## Router Pattern

##### Load Environment variables

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
tavily_api_key = os.getenv('TAVILY_API_KEY')
model_id = os.getenv('MODEL_ID')
aws_region = os.getenv('AWS_REGION')
bedrock_kb_id = os.getenv('BEDROCK_KB_ID')

##### Model

In [ ]:
## Model - Agent Brain
from langchain_aws import ChatBedrock
llm = ChatBedrock(model=model_id)

##### State - The schema of the State will be the input schema to all Nodes and Edges in the graph

In [ ]:
## Graph State
from typing import TypedDict

class State(TypedDict):
    query: str
    route_path: str
    web_search: str
    kb_search: str
    final_blog: str

##### Tools

In [ ]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults

def search_web(state: State):
    print("SEARHING WEB")
    search_tool = TavilySearchResults(max_results=2)
    web_results = search_tool.invoke(state["query"])
    return {"web_search": web_results}

In [ ]:
import boto3
def query_knowledge_base(state: State):
    """Query the knowledge base for information related to Agents and Agentic workflow
    
    Args:
        query: The query string to search for
    """
    bedrock_agent = boto3.client('bedrock-agent-runtime', region_name = aws_region)
    print("QUERYING KB")


    response = bedrock_agent.retrieve_and_generate(
        input={
            "text": state["query"]  # Your query text goes here
        },
        retrieveAndGenerateConfiguration={
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                "knowledgeBaseId": bedrock_kb_id,
                "modelArn": model_id,
                "retrievalConfiguration": {
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            }
        }
    )

    kb_results = response['output']['text']
    return {"kb_search" : kb_results}

In [ ]:
def blogger(state: State):
    print("WRITING BLOG")
    if 'kb_search' in state:
        prompt = f""" Your job is to create a blog title and a one paragraph blog from this content: {state['kb_search']}"""
    else:
        prompt = f""" Your job is to create a blog title and a one paragraph blog from this content: {state['web_search']}"""
    final_answer = llm.invoke(prompt)           
    return {"final_blog": final_answer.content}   

##### Router

In [ ]:
def route_node(state:State):
    print("ROUTER")
    prompt = f""" Your job is to identify the topic of the query in {state['query']} if the topic is about agents, answer "agent" otherwise answer as "other".  just provide the answer.  ignore any preambles"""
    route_decision = llm.invoke(prompt)           
    return {"route_path": route_decision.content}   

In [ ]:
def route_decision(state: State):
    if state['route_path'] == 'agent':
        return "kb"
    else:
        return "web"

##### Graph

In [ ]:
from langgraph.graph import StateGraph, START, END

# Build workflow
router = StateGraph(State)

# Add nodes
router.add_node("route", route_node)
router.add_node("search_kb", query_knowledge_base)
router.add_node("search_web", search_web)
router.add_node("blog_writer", blogger)

# Add edges to connect nodes
router.add_edge(START, "route")
router.add_conditional_edges("route", route_decision, {"kb": "search_kb", "web" : "search_web"})
router.add_edge("search_kb", "blog_writer")
router.add_edge("search_web", "blog_writer")

router.add_edge("blog_writer", END)

##### Compile Graph

In [ ]:
agent = router.compile()

##### Display Graph

In [ ]:
import requests
from IPython.display import Image, display

requests.adapters.DEFAULT_TIMEOUT = 30  # Increase from default 10 seconds
display(Image(agent.get_graph().draw_mermaid_png()))

##### Invoke Agent

In [ ]:
state = agent.invoke({"query": "who is usain bolt?"})
state['final_blog']